In [1]:
import influxdb
from influxdb import InfluxDBClient, DataFrameClient
import pandas as pd

In [2]:
client = InfluxDBClient('localhost', 8086, 'root', 'root', 'mydb')

In [3]:
result = client.query('select * from tag1;')

In [4]:
print("Result: {0}".format(result))

Result: ResultSet({'('tag1', None)': [{'time': '2020-07-08T18:21:48.201433Z', 'area': 'area1', 'plant': 'plant1', 'unit': 'kg/h', 'value': 64.6}, {'time': '2020-07-08T21:08:45.900390Z', 'area': 'area1', 'plant': 'plant1', 'unit': 'kg/h', 'value': 65.6}, {'time': '2020-07-08T21:08:51.524166Z', 'area': 'area1', 'plant': 'plant1', 'unit': 'kg/h', 'value': 65.7}, {'time': '2020-07-08T21:08:57.189089Z', 'area': 'area1', 'plant': 'plant1', 'unit': 'kg/h', 'value': 63.7}, {'time': '2020-07-08T21:09:03.434352Z', 'area': 'area1', 'plant': 'plant1', 'unit': 'kg/h', 'value': 68.7}, {'time': '2020-07-08T21:09:08.543839Z', 'area': 'area1', 'plant': 'plant1', 'unit': 'kg/h', 'value': 68.8}]})


In [5]:
# extract metadata from tagname (filename)
fn = "POARTI-17934_av.csv"
fp = 'F:\\Data Science\\04 Benzene Extraction\\'
loc = fn[:2]
plant = fn[2:4]
tag = fn.split(plant)[1].split("_")[0]
tagtype = fn.split("_")[1].split(".")[0].upper()
f = fp+fn
print(loc,plant,tag,tagtype,f)

PO AR TI-17934 AV F:\Data Science\04 Benzene Extraction\POARTI-17934_av.csv


In [6]:
%%time
# read csv file with extracted data from IP21
# convert csv to pandas line input for influxdb
df = pd.read_csv(f,index_col=0)
df['epoch'] = pd.to_datetime(df['TS_START'])#.astype('int64')//1e9
df.set_index('epoch',inplace=True)
df.drop('TS_START',axis=1,inplace=True)
df.columns = ['value']
df['TAG'] = tag
df['TYPE'] = tagtype
df['LOC'] = loc
df['PLANT'] = plant
df

Wall time: 14.6 s


,value,TAG,TYPE,LOC,PLANT
epoch,,,,,
2014-01-01 00:00:00,0.000000,TI-17934,AV,PO,AR
2014-01-01 00:15:00,0.000000,TI-17934,AV,PO,AR
2014-01-01 00:30:00,0.000000,TI-17934,AV,PO,AR
2014-01-01 00:45:00,0.000000,TI-17934,AV,PO,AR
2014-01-01 01:00:00,0.000000,TI-17934,AV,PO,AR
...,...,...,...,...,...
2016-11-24 22:45:00,183.180386,TI-17934,AV,PO,AR
2016-11-24 23:00:00,183.241152,TI-17934,AV,PO,AR
2016-11-24 23:15:00,183.052796,TI-17934,AV,PO,AR


In [7]:
%%time
# write dataframe to influxdb
measurement = "IP21"
field_columns = ['value']
tag_columns = ['TAG','TYPE','LOC','PLANT']

dfc = DataFrameClient('localhost', 8086, 'root', 'root', 'mydb')
dfc.write_points(df,measurement=measurement, field_columns=field_columns, tag_columns=tag_columns)

Wall time: 2.82 s


True

In [8]:
#client.drop_measurement("IP21")